In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("../input/tabular-playground-series-feb-2021/train.csv")
test_data = pd.read_csv("../input/tabular-playground-series-feb-2021/test.csv")

In [ ]:
features = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Use as many lines of code as you need!
OH = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_cols_train = pd.DataFrame(OH.fit_transform(train_data[features]))
OH_cols_valid = pd.DataFrame(OH.transform(test_data[features]))

# One-hot encoding removed index; put it back
OH_cols_train.index = train_data.index
OH_cols_valid.index = test_data.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train_data.drop(features, axis=1)
num_X_valid = test_data.drop(features, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

In [ ]:
y_train = OH_X_train.target

X_train = OH_X_train.copy()
X_train.drop(['target'], axis=1, inplace=True)

X_valid = OH_X_valid.copy()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# function for comparing different approaches
model = RandomForestRegressor(n_estimators=200, random_state=0)
model.fit(X_train, y_train)
preds = model.predict(X_valid)

In [ ]:
from xgboost import XGBRegressor

# Define the model
XGBoost_model = XGBRegressor(n_estimators=1000, learning_rate=0.05) #Here

# Fit the model
XGBoost_model.fit(X_train, y_train)

# Get predictions
predictions = XGBoost_model.predict(X_valid)

In [ ]:
output = pd.DataFrame({'id': test_data.id, 'target': predictions})
output.to_csv('submission (XGBoost).csv', index=False)